In [9]:
import os
import random
import gc
import argparse
import numpy as np
import h5py as h5
import torch
import torch.nn.functional as F
import wandb

from models.autoencoder import Autoencoder 
from losses.cyl_ptpz_mae import CylPtPzMAE

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

In [10]:
#setting a seed like in ae_legacy
def set_seed(seed=123):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [11]:
def distance_corr(var_1,var_2,normedweight,power=1):
    """var_1: First variable to decorrelate (eg mass)
    var_2: Second variable to decorrelate (eg classifier output)
    normedweight: Per-example weight. Sum of weights should add up to N (where N is the number of examples)
    power: Exponent used in calculating the distance correlation
    
    va1_1, var_2 and normedweight should all be 1D torch tensors with the same number of entries
    
    Usage: Add to your loss function. total_loss = BCE_loss + lambda * distance_corr
    """ 
    
    xx = var_1.view(-1, 1).repeat(1, len(var_1)).view(len(var_1),len(var_1))
    yy = var_1.repeat(len(var_1),1).view(len(var_1),len(var_1))
    amat = (xx-yy).abs()

    xx = var_2.view(-1, 1).repeat(1, len(var_2)).view(len(var_2),len(var_2))
    yy = var_2.repeat(len(var_2),1).view(len(var_2),len(var_2))
    bmat = (xx-yy).abs()

    amatavg = torch.mean(amat*normedweight,dim=1)
    Amat=amat-amatavg.repeat(len(var_1),1).view(len(var_1),len(var_1))\
        -amatavg.view(-1, 1).repeat(1, len(var_1)).view(len(var_1),len(var_1))\
        +torch.mean(amatavg*normedweight)

    bmatavg = torch.mean(bmat*normedweight,dim=1)
    Bmat=bmat-bmatavg.repeat(len(var_2),1).view(len(var_2),len(var_2))\
        -bmatavg.view(-1, 1).repeat(1, len(var_2)).view(len(var_2),len(var_2))\
        +torch.mean(bmatavg*normedweight)

    ABavg = torch.mean(Amat*Bmat*normedweight,dim=1)
    AAavg = torch.mean(Amat*Amat*normedweight,dim=1)
    BBavg = torch.mean(Bmat*Bmat*normedweight,dim=1)

    if(power==1):
        dCorr=(torch.mean(ABavg*normedweight))/torch.sqrt((torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight)))
    elif(power==2):
        dCorr=(torch.mean(ABavg*normedweight))**2/(torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight))
    else:
        dCorr=((torch.mean(ABavg*normedweight))/torch.sqrt((torch.mean(AAavg*normedweight)*torch.mean(BBavg*normedweight))))**power
    
    return dCorr

In [12]:
#calculates anomaly score like in ae legacy
def distance_pt(model_ae, data_np, device):
    x = torch.tensor(data_np, dtype=torch.float32, device=device)
    z_mean, z_logvar, _ = model_ae.encoder(x)
    score = torch.sum(z_mean**2, dim=1)
    return score.detach().cpu().numpy()

In [13]:
#function to make 2D histograms
def make_2D_hist(x, y, xlabel, ylabel, title, wandb_key, bins=40):
    fig = plt.figure(figsize=(5,4))
    plt.hist2d(x, y, bins=bins)
    plt.colorbar(label='Counts')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    wandb.log({wandb_key: wandb.Image(fig)})
    plt.close(fig)

In [14]:
#prints h5 tree to examine
def print_h5_tree(h, prefix=""):
    for k in h.keys():
        item = h[k]
        if hasattr(item, 'keys'):
            print(prefix + f"[GROUP] {k}")
            print_h5_tree(item, prefix + "  ")
        else:
            try:
                print(prefix + f"{k}: shape={item.shape}, dtype={item.dtype}")
            except Exception:
                print(prefix + f"{k}: <dataset>")

In [15]:
def run(config):
    #set seed
    seed = 123
    set_seed(seed)

    #move to gpu if avail
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f"Using: {device}")

    #login to wandb
    print("Logging in to wandb...")
    wandb.login(key="24d1d60ce26563c74d290d7b487cb104fc251271")
    wandb.init(project="Double Disco Axo Training",
               settings=wandb.Settings(_disable_stats=True),
               config=config)
    run_name = wandb.run.name
    print(f"Run name: {run_name}")

    #scaling
    alpha = float(config['alpha'])
    ae_lr = float(config['ae_lr'])

    #load data
    print("Loading dataset...")
    fpath = '/axovol/training/v5/conditionsupdate_apr25.h5'
    with h5.File(fpath, 'r') as f:
        root = f['data'] if 'data' in f else f

        #print h5 tree to view
        # print("Printing h5 tree...")
        # print_h5_tree(root)
    
        x_train = root['Background_data']['Train']['DATA'][:]
        x_test = root['Background_data']['Test']['DATA'][:]
        print(f"Train shape: {x_train.shape}, Test shape: {x_test.shape}")
    
        #flatten per event
        x_train_bkg = x_train.reshape(x_train.shape[0], -1)
        x_test_bkg  = x_test.reshape(x_test.shape[0], -1)
    
        scale = root['Normalisation']['norm_scale'][:]
        bias = root['Normalisation']['norm_bias'][:]
    
        l1_bits_bkg_test = root['Background_data']['Test']['L1bits'][:]
    
        #load signal data
        SIGNAL_NAMES = list(root['Signal_data'].keys())
        signal_data_dict = {}
        signal_l1_dict = {}
        for sname in SIGNAL_NAMES:
            x_sig = root['Signal_data'][sname]['DATA'][:]
            x_sig = x_sig.reshape(x_sig.shape[0], -1)
            l1_bits = root['Signal_data'][sname]['L1bits'][:]
            signal_data_dict[sname] = x_sig
            signal_l1_dict[sname] = l1_bits

    print("Data finished loading.")
    
    dataset = x_train_bkg
    dataset_test = x_test_bkg

    #should be 57
    features = dataset.shape[1]


    print("Building mask...")
    # same mask from ae legacy
    mask_dict = {
        "MET":[True],
        "EGAMMA":[True,True,True,True,False,False,False,False,False,False,False,False],
        "MUON":[True,True,True,True,False,False,False,False],
        "JET":[True,True,True,True,True,True,True,True,True,True,False,False]
    }
    
    #build cyl_ptpz_mae mask (input scales and biases)
    reco_loss_fn = CylPtPzMAE(scale, bias).to(device)
    print("Mask is ready.")


    #ae config
    latent_dim = int(config['ae_latent'])
    enc_nodes = list(config['ae_nodes'])
    dec_nodes = [24, 32, 64, 128, features] 

    ae_cfg = {
        "features": features,
        "latent_dim": latent_dim,
        "encoder_config": {"nodes": enc_nodes},
        "decoder_config": {"nodes": dec_nodes},
        "alpha": alpha
    }

    #put ae on device/init
    ae_1 = Autoencoder(ae_cfg).to(device)
    ae_2 = Autoencoder(ae_cfg).to(device)
    print("Autoencoders are ready.")

    #optimizer (adam)
    optimizer = torch.optim.Adam(list(ae_1.parameters()) + list(ae_2.parameters()), lr=ae_lr)

    #cosine restarts
    warmup_epochs = 10
    cos = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=32, T_mult=2, eta_min=0.0)

    #sets learning rate
    def set_lr(lr):
        for g in optimizer.param_groups:
            g['lr'] = lr

    #hyperparameters
    Epochs_AE = 50
    Batch_size = 16384

    #get disco param from config, else set to 1
    lambda_disco = float(config.get("lambda_disco", 1))

    print("Moving data to device...")
    X = torch.tensor(dataset, dtype=torch.float32, device=device)
    print("Data on device.")

    #training loop
    print("Starting the training loop!")
    N = X.size(0)
    for epoch in range(Epochs_AE):

        #init lists for 2D plotting
        ae1_reco_loss = []
        ae2_reco_loss = []
        
        #cosine warmup step (sets the learning rate based on cosine schedule)
        if epoch < warmup_epochs:
            lr = ae_lr * (epoch + 1) / warmup_epochs
            set_lr(lr)
        else:
            cos.step(epoch - warmup_epochs)

        #shuffles data incides for each epoch
        perm = torch.randperm(N, device=device)

        #init losses
        total_loss = total_reco1 = total_reco2 = total_disco = 0.0

        #loops over dataset in steps of batch sizze
        for i in range(0, N, Batch_size):
            #picks indices for current batch
            idx = perm[i:i+Batch_size]

            #selects batch of samples from data set X
            xb = X[idx]

            #ae 1
            recon1, z1 = ae_1(xb)
            
            #ae 2
            recon2, z2 = ae_2(xb)

            #get reco loss from custom func
            reco1_per = reco_loss_fn(recon1, xb)
            reco2_per = reco_loss_fn(recon2, xb)

            #from paper code weight
            B = xb.shape[0]
            w = torch.ones(B, device=reco1_per.device, dtype=reco1_per.dtype)

            #disco loss (ask Melissa about since using mu instead of z)
            #disco = disco_loss(mu1, mu2)
            disco = distance_corr((reco1_per), (reco2_per), w, power=1)

            #same scaling from ae legacy
            reco1 = ae_1.alpha*reco1_per.mean()
            reco2 = ae_2.alpha*reco2_per.mean()

            #calc total loss (scalar)
            loss = reco1 + reco2 + lambda_disco * disco

            #zero grads
            optimizer.zero_grad(set_to_none=True)

            #backward pass
            loss.backward()

            #do some gradient clipping
            torch.nn.utils.clip_grad_norm_(list(ae_1.parameters()) + list(ae_2.parameters()), max_norm=5.0)
            optimizer.step()

            #add losses to list for wandb plotting
            total_loss+=loss.item()
            total_reco1+=reco1.item()
            total_reco2+=reco2.item()
            total_disco+=disco.item()

            #add loss to list for 2d plotting
            ae1_reco_loss.append(reco1.item())
            ae2_reco_loss.append(reco2.item())

        print(f"[EPOCH {epoch}/{Epochs_ae}] "
          f"Loss={total_loss:.4f} "
          f"Reco1={total_reco1:.4f} Reco2={total_reco2:.4f} "
          f"DisCo={total_disco:.4f}")

        #log in wandb
        wandb.log({
            "Epochae": epoch,
            "TotalLossae": total_loss,
            "RecoLossae1": total_reco1,
            "RecoLossae2": total_reco2,
            "DisCoLoss": total_disco,
        })

        #convert to np array for plot
        ae1_reco_np = np.array(ae1_reco_loss)
        ae2_reco_np = np.array(ae2_reco_loss)

        make_2D_hist(ae1_reco_np, ae2_reco_np,
                    "Reco Loss (ae1)", "Reco Loss (ae2)",
                    f"Epoch {epoch}: Reco ae1 vs Reco ae2",
                    wandb_key="Hists2D/Reco_ae1_vs_Reco_ae2")

    

    print("Finished training.")

    #save models
    torch.save(ae_1.state_dict(), "ae1_trained.pth")
    torch.save(ae_2.state_dict(), "ae2_trained.pth")




In [16]:
config = {
    'ae_lr': 1e-4,
    'alpha': 0.5,
    'ae_latent': 8,
    'ae_nodes': [28, 14],
    'lambda_disco': 10000.0
}

run(config)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Using: cuda:0
Logging in to wandb...


Run name: fanciful-smoke-130
Loading dataset...
Train shape: (1999965, 19, 3), Test shape: (4511092, 19, 3)
Data finished loading.
Building mask...
Mask is ready.
Autoencoders are ready.
Moving data to device...
Data on device.
Starting the training loop!


OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 10.90 GiB of which 179.25 MiB is free. Process 614511 has 10.72 GiB memory in use. Of the allocated memory 10.08 GiB is allocated by PyTorch, and 500.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#reload data same way as in traiining
print("Loading dataset...")
fpath = '/axovol/training/v5/conditionsupdate_apr25.h5'
with h5.File(fpath, 'r') as f:
    root = f['data'] if 'data' in f else f

    x_train = root['Background_data']['Train']['DATA'][:]
    x_test  = root['Background_data']['Test']['DATA'][:]
    print(f"Train shape: {x_train.shape}, Test shape: {x_test.shape}")

    #flatten per event
    x_train_bkg = x_train.reshape(x_train.shape[0], -1)
    x_test_bkg  = x_test.reshape(x_test.shape[0], -1)

    scale = root['Normalisation']['norm_scale'][:]
    bias  = root['Normalisation']['norm_bias'][:]

    l1_bits_bkg_test = root['Background_data']['Test']['L1bits'][:]

    print("Available signal names:")
    print(list(root['Signal_data'].keys()))

    #load signal data
    x_sig = root['Signal_data']['GluGluHToTauTau']['DATA'][:]
    x_sig = x_sig.reshape(x_sig.shape[0], -1)
    l1_bits_sig = root['Signal_data']['GluGluHToTauTau']['L1bits'][:]

print("Data finished loading.")

features = x_test_bkg.shape[1]

# reload config based on training
cfg = {
    "features": features,
    "latent_dim": 8,         
    "encoder_config": {"nodes": [28, 14]}, 
    "decoder_config": {"nodes": [24, 32, 64, 128, features]}, 
    "alpha": 0.5,
    "beta": 0.5
}

ae_1 = ae(cfg).to(device)
ae_2 = ae(cfg).to(device)

#load model weights
ae_1.load_state_dict(torch.load("ae1_trained.pth", map_location=device))
ae_2.load_state_dict(torch.load("ae2_trained.pth", map_location=device))
ae_1.eval()
ae_2.eval()

#compute anomaly scores
bkg_scores_1 = distance_pt(ae_1, x_test_bkg, device)
sig_scores_1 = distance_pt(ae_1, x_sig, device)

bkg_scores_2 = distance_pt(ae_2, x_test_bkg, device)
sig_scores_2 = distance_pt(ae_2, x_sig, device)

#ae 1 anomaly score plot
plt.figure(figsize=(6,4))
bins = np.linspace(np.percentile(bkg_scores_1, 0.5), np.percentile(bkg_scores_1, 99.5), 60)
plt.hist(bkg_scores_1, bins=bins, alpha=0.5, label='Background')
plt.hist(sig_scores_1, bins=bins, alpha=0.5, label="GluGluHToTauTau")
plt.xlabel("Anomaly score")
plt.ylabel("Events")
plt.legend()
plt.title("ae 1 Anomaly Scores")
plt.yscale('log')
plt.show()

#ae 1 ROC curve
y_true  = np.concatenate([np.zeros_like(bkg_scores_1), np.ones_like(sig_scores_1)])
y_score = np.concatenate([bkg_scores_1, sig_scores_1])
fpr, tpr, _ = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, lw=2, label=f"AUC = {roc_auc:.3f}")
plt.plot([0,1],[0,1],'--', lw=1, color='gray')
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title(f"ROC ae 1")
plt.legend()
plt.show()


#ae 2 anomaly score plot
plt.figure(figsize=(6,4))
bins = np.linspace(np.percentile(bkg_scores_2, 0.5), np.percentile(bkg_scores_2, 99.5), 60)
plt.hist(bkg_scores_2, bins=bins, alpha=0.6, label='Background')
plt.hist(sig_scores_2, bins=bins,alpha=0.6, label="GluGluHToTauTau")
plt.xlabel("Anomaly score")
plt.ylabel("Events")
plt.legend()
plt.title("ae 2 anomaly scores")
plt.yscale('log')
plt.show()

#ae 2 ROC curve plot
y_true  = np.concatenate([np.zeros_like(bkg_scores_2), np.ones_like(sig_scores_2)])
y_score = np.concatenate([bkg_scores_2, sig_scores_2])
fpr, tpr, _ = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr, lw=2, label=f"AUC = {roc_auc:.3f}")
plt.plot([0,1],[0,1],'--', lw=1, color='gray')
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title(f"ROC ae 2")
plt.legend()
plt.show()